# PyConJP2022
## 公的統計APIデータ取得PythonライブラリJPy-DataReader

In [ ]:
%%HTML
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>

# e-Stat API機能の事前準備

e-Stat API機能に事前に[ユーザー登録](https://www.e-stat.go.jp/mypage/user/preregister)して、アプリケーションIDをを取得します。

In [ ]:
api_key = "Your_Application_ID" # 取得したアプリケーションID　

# e-Stat API機能でのデータ取得

statsDataIdは「統計表情報取得」で得られる統計表IDです。e-Statの[データベース](https://www.e-stat.go.jp/stat-search/database?page=1)から検索できます。「政府統計コード」ではなく、「統計表表示 ID」です。

## requestsでのデータ取得

まず、JPy-DataReaderを使わず、requestsでデータ取得してみます。

In [ ]:
!pip install requests

In [ ]:
import requests

url = "https://api.e-stat.go.jp/rest/3.0/app/json/getStatsData?"
params = {
    "appId": api_key,
    "statsDataId": "0003000795",
}

res = requests.get(url, params)
out = res.json()

out

JSONの中身を見てみます。

In [ ]:
out.keys()

In [ ]:
out['GET_STATS_DATA'].keys()

In [ ]:
out['GET_STATS_DATA']['STATISTICAL_DATA'].keys()

In [ ]:
out['GET_STATS_DATA']['STATISTICAL_DATA'][ 'DATA_INF'].keys()

統計データ自体は以下に格納されています。

In [ ]:
out['GET_STATS_DATA']['STATISTICAL_DATA'][ 'DATA_INF']['VALUE']

pandasのデータフレーム形式で読み込みます。

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

df = pd.DataFrame(out['GET_STATS_DATA']['STATISTICAL_DATA'][ 'DATA_INF']['VALUE'])
df

VALUEだけでは、それぞれの列が何を表しているのかわかりません。各列のメタデータはCLASS_INFに格納されており、列ID名で結合することができます。

In [ ]:
out['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

In [ ]:
pd.DataFrame(out['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ'][1][ 'CLASS'])

値の列「$」を見ると、数値でない欠損と思われる特殊文字があります。

In [ ]:
sorted(df['$'].unique().tolist())

特殊文字はNOTEで、どのような種類があるかとその意味を調べることができます。

In [ ]:
out['GET_STATS_DATA']['STATISTICAL_DATA'][ 'DATA_INF']['NOTE']

# JPy-DataReaderでデータ取得

e-Statなど公的統計APIからデータを取得するPythonのライブラリJPy-DataReaderを作りました。

In [ ]:
!pip install jpy-datareader

さくっとデータ取得する場合、以下の２行でデータ取得とデータ加工ができます。

In [ ]:
import jpy_datareader as jdr
jdr.get_data_estat_statsdata(api_key, statsDataId="0003000795")

以下も同じ挙動です。

In [ ]:
import jpy_datareader.data as web
web.DataReader("0003000795", "estat", api_key=api_key)

よくデータ取得に使われる[pandas-datareader](https://pandas-datareader.readthedocs.io/en/latest/)に仕様を似せています。

In [ ]:
!pip install pandas-datareader

In [ ]:
import pandas_datareader as pdr
pdr.get_data_fred('GS10').tail(10)  # 10年満期米国債市場金利

In [ ]:
import pandas_datareader.data as web
web.DataReader('TUD', 'oecd')  # 労働組合データ

In [ ]:
from pandas_datareader import fred
fred.FredReader('GS10').read()

もう少し細かく設定してデータ取得したい場合には、以下を使います。

In [ ]:
from jpy_datareader import estat
estat.StatsDataReader(api_key, statsDataId="0003000795").read()

In [ ]:
df

# JPy-DataReaderのサンプルコード

### 人口推計のデータを扱いたいとき

In [ ]:
from jpy_datareader import estat
df = estat.StatsDataReader(api_key, statsDataId="0003448228").read()
cond = df["男女別・性比名"] == "男女計"
cond &= df["人口名"] == "日本人人口"
cond &= df["年齢各歳名"] != "総数"
cond &= df["時間軸（年月日現在）名"] == "2021年10月1日現在"
population = df[cond][["分類03_コード", "年齢各歳名", "値"]]
population["年齢"] = population["分類03_コード"].astype(int) - 1001
working_age_population = int(population[(population["年齢"]>=15)&(population["年齢"]<65)]["値"].sum())
print("生産年齢人口: ", working_age_population, ", 生産年齢除く人口: ", int(population["値"].sum() - working_age_population))

分割して見ていきます。

人口推計のデータを取得します。

In [ ]:
from jpy_datareader import estat
df = estat.StatsDataReader(api_key, statsDataId="0003448228").read()

条件指定して必要なデータを抽出します。int型の年齢は、メタデータから「歳」を取り除くことでもできますが、ここでは分類コードを利用して出力しています。

In [ ]:
cond = df["男女別・性比名"] == "男女計"
cond &= df["人口名"] == "日本人人口"
cond &= df["年齢各歳名"] != "総数"
cond &= df["時間軸（年月日現在）名"] == "2021年10月1日現在"
population = df[cond][["分類03_コード", "年齢各歳名", "値"]]
population["年齢"] = population["分類03_コード"].astype(int) - 1001

日本の人口は約1億2千6百万人。では、15〜64歳の生産年齢人口とそれ以外の人口、どちらが多いか(2021年)を計算してみます。

In [ ]:
working_age_population = int(population[(population["年齢"]>=15)&(population["年齢"]<65)]["値"].sum())
print("生産年齢人口: ", working_age_population, ", 生産年齢除く人口: ", int(population["値"].sum() - working_age_population))

### 経済センサスを扱いたいとき

In [ ]:
from jpy_datareader import estat
statsdata = estat.StatsDataReader(api_key, statsDataId="0003449721", 
    cdTab="202-2021", cdArea="00000",
    cdCat02="00", cdCat03="00")
df = statsdata.read(normal=True)
cond = df["単一・複数名"] == "総数"
cond &= (df["企業産業大分類名"] == "情報通信業") | (df["企業産業大分類名"] == "医療，福祉")
df[cond][["企業産業大分類名", "値"]]

データ数が多いとき(10万件以上目安)はメタ情報を確認して、分割して必要なデータのみ取得するのをおすすめします。

In [ ]:
metainfo = estat.MetaInfoReader(api_key, statsDataId="0003449721", name_or_id="name")
meta = metainfo.read()
metainfo.OVERALL_TOTAL_NUMBER

各属性のメタ情報をDataFrameの辞書で取得しています。

In [ ]:
meta.keys()

条件指定する際の各コード値の意味を調べます。

In [ ]:
meta['tab']

条件を指定してデータ取得します

In [ ]:
statsdata = estat.StatsDataReader(api_key, statsDataId="0003449721", 
    cdTab="202-2021", cdArea="00000",
    cdCat02="00", cdCat03="00")
df = statsdata.read(normal=True)

日本の企業数は、「情報通信業」と「医療，福祉業」だとどちらが多いかを見てみます。

In [ ]:
cond = df["単一・複数名"] == "総数"
cond &= (df["企業産業大分類名"] == "情報通信業") | (df["企業産業大分類名"] == "医療，福祉")
df[cond][["企業産業大分類名", "値"]]

### 家計調査を扱いたいとき

In [ ]:
import jpy_datareader.data as web
df = web.DataReader("0002210018", "estat", api_key=api_key)
cond = df["貯蓄・負債"] == "貯蓄"
cond &= df["世帯区分"] == "二人以上の世帯（2000年～）"
cond &= df["世帯主の年齢階級"] == "60～69歳"
cond &= df["世帯の負債有無"] == "全世帯"
cond &= df["時間軸（年次）"] == "2021年"
df[cond]

分割して見ていきます。
家計調査 貯蓄・負債編のデータを取得します。

In [ ]:
import jpy_datareader.data as web
df = web.DataReader("0002210018", "estat", api_key=api_key)
df

60～69歳の平均貯蓄額は3,000万円を超えているか(2021年)を調べてみます。
条件指定して必要なデータを抽出します。

In [ ]:
cond = df["貯蓄・負債"] == "貯蓄"
cond &= df["世帯区分"] == "二人以上の世帯（2000年～）"
cond &= df["世帯主の年齢階級"] == "60～69歳"
cond &= df["世帯の負債有無"] == "全世帯"
cond &= df["時間軸（年次）"] == "2021年"
df[cond]

# おまけ:plotlyで可視化してみる

In [ ]:
!pip install plotly

In [ ]:
import plotly
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
statsdata = estat.StatsDataReader(api_key, statsDataId="0003449721", 
    cdTab="202-2021", cdCat02="00", cdCat03="00")
df = statsdata.read(normal=True)
df = df[df["単一・複数名"] == "総数"]

In [ ]:
df["都道府県"] = df["地域_コード"].str[:2] + df["地域名"]
ecensus_df = df[["企業産業大分類名", "都道府県","値"]]
cond = (ecensus_df["企業産業大分類名"] != "全産業（S_公務を除く）") & (ecensus_df["企業産業大分類名"] != "非農林漁業（S_公務を除く）")
cond &= ecensus_df["都道府県"] != "00全国"
ecensus_df = ecensus_df[cond]

In [ ]:
fig = px.bar(ecensus_df, 
             x='都道府県',
             y='値', 
             color='企業産業大分類名',
             )
fig